In [7]:
import ast

In [6]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(0, '../scoring_0_7_3')
import time
import datetime
import operator
import math
import random
import numpy as np
import pandas as pd
import cx_Oracle
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
import pickle
import gc
from tqdm.notebook import tqdm
import xgboost as xgb
import scoring
from scipy.special import logit, expit

/opt/anaconda3/envs/dl_env/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [8]:
sns.set()
%matplotlib inline
%config InlineBackend.close_figures=True
from IPython.display import display, Markdown
pd.options.display.max_columns = None
pd.options.display.max_rows = 15
output_folder = 'FEATURE_ENGINEERING'

if not os.path.exists(output_folder): os.makedirs(output_folder)
if not os.path.exists(output_folder+'/performance'): os.makedirs(output_folder+'/performance')
if not os.path.exists(output_folder+'/predictors'): os.makedirs(output_folder+'/predictors')
if not os.path.exists(output_folder+'/stability'): os.makedirs(output_folder+'/stability')
if not os.path.exists(output_folder+'/stability_short'): os.makedirs(output_folder+'/stability_short')
if not os.path.exists(output_folder+'/analysis'): os.makedirs(output_folder+'/analysis')
if not os.path.exists(output_folder+'/model'): os.makedirs(output_folder+'/model')
if not os.path.exists(output_folder+'/nan_share'): os.makedirs(output_folder+'/nan_share')
    
psw_version = '0.8.3'
scoring.check_version(psw_version)

In [9]:
from tqdm import tqdm  #
from scoring import doctools

documentation = doctools.ProjectParameters()

In [11]:
data

,index,flag_bad,de_age,de_gender,de_date_joined,de_num_friends,de_monthly_salary,de_employment_type,de_employment_duration,de_accomodation_type,de_education,de_marital_status,de_children,ph_total_contacts,ph_other_device_info,ph_call_log_stats,ph_country_code,ph_app_list,fb_gender,fb_dob,fb_last_updated_date,fb_relation
0,0,0,26,1,2018-08-04,2535,3000000,1,51,4,2,2,0,66,Xiaomi,"{""percent_incoming_nighttime"": 1.83, ""percent_...",id,"['com.google.android.music', 'com.facebook.orc...",male,11/23/1991,2017-12-21T04:38:00+0000,Married
1,1,0,22,1,2018-02-01,685,4000000,1,28,4,4,1,0,69,OPPO,"{""percent_incoming_nighttime"": 0.0, ""percent_o...",id,"['com.google.android.instantapps.supervisor', ...",male,01/20/1995,2017-11-25T17:23:09+0000,NaN
2,2,1,32,1,2018-01-01,1765,7000000,1,35,4,4,2,2,315,OPPO,"{""percent_incoming_nighttime"": 6.65, ""percent_...",id,"['com.tencent.ibg.joox', 'com.mrgames.larvaact...",male,10/13/1985,2017-12-23T18:05:57+0000,Married
3,3,0,31,2,2018-04-01,407,20000000,3,37,4,3,2,2,84,OPPO,"{""percent_incoming_nighttime"": 0.0, ""percent_o...",id,"['com.kfit.fave', 'app.bpjs.mobile', 'com.redd...",female,07/27/1986,2017-12-05T09:37:19+0000,Married
4,4,0,27,1,2018-02-01,3064,3000000,1,36,4,3,1,0,0,vivo,"{""percent_incoming_nighttime"": 0.0, ""percent_o...",id,"['com.lenovo.anyshare.gps', 'com.UCMobile.intl...",male,07/05/1990,2017-12-03T09:09:48+0000,In a relationship
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41017,41017,0,27,2,2018-11-07,997,5000000,2,51,4,2,2,2,1026,OPPO,"{""percent_incoming_nighttime"": 5.66, ""percent_...",id,"['com.instagram.android', 'com.google.android....",NaN,NaN,NaN,NaN
41018,41018,0,26,1,2018-11-07,955,3000000,2,24,4,2,1,0,0,Xiaomi,"{""percent_incoming_nighttime"": 10.71, ""percent...",id,"['com.youmusic.magictiles', 'com.ayopop', 'com...",NaN,NaN,NaN,NaN
41019,41019,0,22,2,2018-11-07,881,3000000,1,9,4,4,1,0,272,samsung,"{""percent_incoming_nighttime"": 0.0, ""percent_o...",id,"['com.grabtaxi.passenger', 'jp.co.translimit.b...",female,10/02/1995,2017-10-25T08:05:13+0000,NaN
41020,41020,0,29,2,2018-11-07,540,6000000,1,51,4,2,1,0,0,Xiaomi,"{""percent_incoming_nighttime"": 1.27, ""percent_...",id,"['cn.wps.moffice_eng', 'com.vuclip.viu', 'com....",NaN,NaN,NaN,NaN


In [10]:
data= pd.read_csv('demographic.csv')
data['de_date_joined']= pd.to_datetime(data['de_date_joined']).dt.strftime('%Y-%m-%d')
data['ph_other_device_info']= data['ph_other_device_info'].str.split(',').str[1].str.split(':').str[1].str.replace('}','').str.replace('"','')

<ipython-input-10-197dfc0a27e6>:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  data= pd.read_csv('demographic.csv')
<ipython-input-10-197dfc0a27e6>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['ph_other_device_info']= data['ph_other_device_info'].str.split(',').str[1].str.split(':').str[1].str.replace('}','').str.replace('"','')


In [13]:
def data_information(df):
    data_info = pd.DataFrame({'Data_Type':df.dtypes,
                              'N_Missing':df.isnull().sum(),
                              '%_Missing':["{0:.0%}".format(x) for x in df.isnull().sum()/len(df)], 
                              'N_Unique':df.nunique().values,
                              'Sample':list(zip(df.sample(5).values[0], df.sample(5).values[4]))})
    return data_info


In [15]:
data_information(data)

,Data_Type,N_Missing,%_Missing,N_Unique,Sample
index,int64,0,0%,41022,"(29765, 10665)"
flag_bad,int64,0,0%,2,"(0, 0)"
de_age,int64,0,0%,50,"(23, 18)"
de_gender,int64,0,0%,2,"(1, 2)"
de_date_joined,object,0,0%,192,"(2018-06-19, 2018-02-21)"
...,...,...,...,...,...
ph_app_list,object,0,0%,40097,"(['com.inditex.zara', 'com.vsco.cam', 'jp.co.c..."
fb_gender,object,1513,4%,2,"(male, female)"
fb_dob,object,9653,24%,8931,"(nan, 02/27/1999)"
fb_last_updated_date,object,1470,4%,39469,"(2018-03-14T11:58:06+0000, 2017-11-17T11:45:21..."


In [786]:
cols_pred=['de_age',
 'de_gender',
 'de_date_joined',
 'de_num_friends',
 'de_monthly_salary',
 'de_employment_type',
 'de_employment_duration',
 'de_accomodation_type',
 'de_education',
 'de_marital_status',
 'de_children',
 'ph_total_contacts',
 'ph_other_device_info',
 'ph_call_log_stats',
 'ph_country_code',
 'ph_app_list',
 'fb_gender',
 'fb_dob',
 'fb_last_updated_date',
 'fb_relation']

In [5]:
# data

In [9]:
data.head(2)

,index,flag_bad,de_age,de_gender,de_date_joined,de_num_friends,de_monthly_salary,de_employment_type,de_employment_duration,de_accomodation_type,de_education,de_marital_status,de_children,ph_total_contacts,ph_other_device_info,ph_call_log_stats,ph_country_code,ph_app_list,fb_gender,fb_dob,fb_last_updated_date,fb_relation
0,0,0,26,1,2018-08-04,2535,3000000,1,51,4,2,2,0,66,Xiaomi,"{""percent_incoming_nighttime"": 1.83, ""percent_...",id,"['com.google.android.music', 'com.facebook.orc...",male,11/23/1991,2017-12-21T04:38:00+0000,Married
1,1,0,22,1,2018-02-01,685,4000000,1,28,4,4,1,0,69,OPPO,"{""percent_incoming_nighttime"": 0.0, ""percent_o...",id,"['com.google.android.instantapps.supervisor', ...",male,01/20/1995,2017-11-25T17:23:09+0000,NaN


In [11]:
new_try= data.copy()

# Feature engineering

## EXPLORE APP LIST

In [14]:
# Function to convert string list to actual list
def str_to_list(s):
    return ast.literal_eval(s)

# Apply the function to convert strings to lists
new_try['expanded_lists'] = new_try['ph_app_list'].apply(str_to_list)

# Initialize an empty list to store rows with original index
rows = []

# Iterate through each row and each list, appending rows with the original index
for index, row in tqdm(new_try.iterrows(), total=new_try.shape[0], desc="Processing Rows"):
    for item in row['expanded_lists']:
        rows.append({'index': index, 'app_package': item})

# Create a new DataFrame from the list of rows
df_expanded = pd.DataFrame(rows)

print(df_expanded)

Processing Rows: 100%|██████████| 41022/41022 [00:03<00:00, 11416.94it/s]


         index                                app_package
0            0                   com.google.android.music
1            0                          com.facebook.orca
2            0          com.MantisGames.DriftandRallyFREE
3            0                              com.apps.MyXL
4            0  com.google.android.instantapps.supervisor
...        ...                                        ...
1617903  41021                      com.instagram.android
1617904  41021                              com.gojek.app
1617905  41021                  com.telkomsel.telkomselcm
1617906  41021                      com.bukalapak.android
1617907  41021                   com.mfashiongallery.emag

[1617908 rows x 2 columns]


In [98]:
['social media','communication','travel','finance','bank','ecommerce','gambling']

['social media',
 'communication',
 'travel',
 'finance',
 'bank',
 'ecommerce',
 'gambling']

In [136]:
import pandas as pd

# Extended application to category mapping
category_mapping = {
    'com.whatsapp': 'Social Media',
    'com.instagram.android': 'Social Media',
    'com.facebook.katana': 'Social Media',
    'com.twitter.android': 'Social Media',
    'com.linkedin.android': 'Social Media',
    'com.snapchat.android': 'Social Media',
    'org.telegram.messenger': 'Social Media',
    'com.pure.indosat.care': 'Communication',
    'com.facebook.orca': 'Social Media',
    'com.viber.voip': 'Communication',
    'com.google.android.apps.translate': 'Productivity',
    'com.google.android.apps.docs': 'Productivity',
    'com.google.android.apps.maps': 'Navigation',
    'com.google.android.apps.drive': 'Productivity',
    'com.google.android.apps.tasks': 'Productivity',
    'com.spotify.music': 'Entertainment',
    'com.netflix.mediaclient': 'Entertainment',
    'com.amazon.mp3': 'Entertainment',
    'com.google.android.youtube': 'Entertainment',
    'com.tiktok': 'Entertainment',
    'com.tiktok.lite': 'Entertainment',
    'com.douyin': 'Entertainment',
    'com.smule.singandroid': 'Entertainment',
    'com.yinshan.program.banda': 'Productivity',
    'com.miui.player': 'Media',
    'com.cleanmaster.mguard': 'Utility',
    'com.bca': 'Financial',
    'com.finaccel.android': 'Financial',
    'com.bankmandiri.mandirionline': 'Financial',
    'com.ubercab': 'Transportation',
    'com.grabtaxi.passenger': 'Transportation',
    'com.gojek.app': 'Transportation',
    'com.tokopedia.tkpd': 'Shopping',
    'com.lazada.android': 'Shopping',
    'com.shopee.id': 'Shopping',
    'com.bukalapak.android': 'Shopping',
    'com.ecommerce.mcommerce': 'Shopping',
    'com.tradeking': 'Financial',
    'com.tunaikita.cashloan': 'Financial',
    'com.mitracomm.jamsostek': 'Financial',
    'com.dana': 'Financial',
    'com.paypal.android.p2pmobile': 'Financial',
    'com.paytm': 'Financial',
    'com.ovo.id': 'Financial',
    'com.google.android.wallet': 'Financial',
    'com.mxtech.videoplayer.ad': 'Media',
    'com.filemanager': 'Utility',
    'com.android.bbk.lockscreen3': 'Utility',
    'com.baidu.searchbox': 'Utility',
    'com.picsart.studio': 'Photography',
    'com.snapseed': 'Photography',
    'com.andi.alquran.id': 'Productivity',
    'com.microsoft.office.excel': 'Productivity',
    'com.microsoft.office.word': 'Productivity',
    'com.microsoft.office.powerpoint': 'Productivity',
    'com.microsoft.office.outlook': 'Productivity',
    'com.google.android.keep': 'Productivity',
    'com.google.android.notes': 'Productivity',
    'com.google.android.apps.plus': 'Productivity',
    'com.naver.line.android': 'Communication',
    'jp.naver.line.android': 'Communication',
    'com.douban.frodo': 'Entertainment',
    'com.opera.mini.native': 'Browser',
    'com.chrome.browser': 'Browser',
    'com.android.browser': 'Browser',
    'com.firefox': 'Browser',
    'com.cleanmaster.mguard': 'Utility',
    'com.google.android.videos': 'Media',
    'com.duokan.phone.remotecontroller': 'Utility',
    'com.telkomsel.telkomselcm': 'Communication',
    'com.telkom.mwallet': 'Financial',
    'com.path': 'Social Media',
    'com.apps.MyXL': 'Utility',
    'com.spotify.music': 'Entertainment',
    'com.google.android.music': 'Entertainment',
    'com.soundcloud.android': 'Entertainment',
    'com.amazon.music': 'Entertainment',
    'com.spotify.music': 'Entertainment',
    'com.spotify.music': 'Entertainment',
    'com.instagram.boomerang': 'Photography',
    'com.google.android.apps.photos': 'Photography',
    'com.linkit.bimatri': 'Productivity',
    'com.supercell.clashofclans': 'Games',
    'com.supercell.brawlstars': 'Games',
    'com.king.candycrushsaga': 'Games',
    'com.candycrushsaga': 'Games',
    'com.niksoftware.snapseed': 'Photography',
    'com.commsource.beautyplus': 'Photography',
    'com.yelp.android': 'Travel & Local',
    'com.trivago.android': 'Travel & Local',
    'com.booking': 'Travel & Local',
    'com.airbnb.android': 'Travel & Local',
    'com.klook.app': 'Travel & Local',
    'com.booking': 'Travel & Local',
    'com.yelp.android': 'Travel & Local',
    'com.google.android.apps.maps': 'Navigation',
    'com.google.android.apps.navigation': 'Navigation',
    'com.samsung.updatecarriermatch': 'Utility',
    'com.msd.JTClient': 'Financial',
    'com.intsig.camscanner': 'Productivity',
    'com.cleanmaster.mguard': 'Utility',
    'com.mitracomm.jamsostek': 'Financial',
    'com.citibank': 'Financial',
    'com.bankmandiri.mandirionline': 'Financial',
    'com.samsung.updatecarriermatch': 'Utility',
    'com.tencent.mm': 'Social Media',
    'com.tencent.ibg.joox': 'Entertainment',
    'com.google.android.youtube': 'Entertainment',
    'com.xiaomi.mipush.sdk': 'Utility',
    'com.yandex.mobile': 'Utility',
    'com.tangbull.phonerepayment': 'Financial',
    'com.kreditpintar': 'Financial',
    'com.google.android.youtube': 'Entertainment',
    'com.intsig.camscanner': 'Productivity',
    'com.snapchat.android': 'Social Media',
    'com.qq.mail': 'Communication',
    'com.mitracomm.jamsostek': 'Financial',
    'com.jd.jr.cdzj': 'Financial',
    'com.youthtools.zcm': 'Entertainment',
    'com.tangbull.phonerepayment': 'Financial',
    'com.amazon.mShop.android.shopping': 'Shopping',
    'com.xiaomi.channel': 'Entertainment',
    'com.filmora.videomaker': 'Entertainment',
    'com.adobe.spark.post': 'Photography',
    'com.google.android.apps.photos': 'Photography',
    'com.microsoft.office.outlook': 'Productivity',
    'com.google.android.docs': 'Productivity',
    'com.citibank': 'Financial',
    'com.mypackage.app': 'Unknown',
}

# List of applications to categorize
applications = [
    'com.finaccel.android', 'com.whatsapp', 'com.google.android.instantapps.supervisor',
    'com.instagram.android', 'com.grabtaxi.passenger', 'com.gojek.app', 'com.lenovo.anyshare.gps',
    'com.lazada.android', 'com.facebook.katana', 'com.tokopedia.tkpd', 'io.silvrr.installment',
    'com.facebook.orca', 'com.shopee.id', 'cn.wps.moffice_eng', 'jp.naver.line.android',
    'com.bukalapak.android', 'com.google.android.play.games', 'com.bbm', 'com.bca',
    'com.UCMobile.intl', 'com.telkomsel.telkomselcm', 'com.tencent.ibg.joox', 'com.mobile.legends',
    'com.duokan.phone.remotecontroller', 'com.mfashiongallery.emag', 'com.app.tokobagus.betterb',
    'com.traveloka.android', 'com.google.android.inputmethod.pinyin', 'com.roidapp.photogrid',
    'com.google.android.apps.translate', 'com.waze', 'com.miui.enbbs', 'com.swiftkey.swiftkeyconfigurator',
    'com.ludo.king', 'com.swiftkey.languageprovider', 'com.twitter.android', 'com.xiaomi.ab',
    'com.ubercab', 'com.touchtype.swiftkey', 'com.miui.klo.bugreport', 'jd.cdyjy.overseas.market.indonesia',
    'com.miui.android.fashiongallery', 'com.facebook.lite', 'com.yahoo.mobile.client.android.mail',
    'lds.cinema21', 'com.telkom.mwallet', 'com.mi.global.bbs', 'blibli.mobile.commerce',
    'com.picsart.studio', 'com.andi.alquran.id', 'com.pure.indosat.care', 'com.path', 'com.apps.MyXL',
    'org.telegram.messenger', 'tv.hooq.android', 'com.linkedin.android', 'id.co.myhomecredit',
    'com.msd.JTClient', 'com.mitracomm.jamsostek', 'com.smule.singandroid', 'com.samsung.updatecarriermatch',
    'ovo.id', 'com.spotify.music', 'com.android.browser', 'com.opera.mini.native', 'com.bpjstku',
    'com.miui.player', 'com.grabtaxi.driver2', 'com.kreditpintar', 'com.google.android.apps.youtube.mango',
    'com.quvideo.xiaoying', 'app.bpjs.mobile', 'com.microsoft.office.excel', 'com.mxtech.videoplayer.ad',
    'com.bankmandiri.mandirionline', 'com.microsoft.office.word', 'com.ss.android.ugc.trill',
    'com.naver.linewebtoon', 'iflix.play', 'com.yinshan.program.banda', 'com.engloryintertech.caping',
    'com.linecorp.b612.android', 'com.vuclip.viu', 'com.google.android.apps.tachyon', 'vStudio.Android.Camera360',
    'com.jobstreet.jobstreet', 'com.google.android.apps.docs', 'com.google.android.apps.photos',
    'com.snapchat.android', 'com.instagram.boomerang', 'com.tangbull.phonerepayment', 'com.king.candycrushsaga',
    'com.rsupport.rs.activity.rsupport.aas2', 'com.linkit.bimatri', 'com.niksoftware.snapseed',
    'com.commsource.beautyplus', 'com.google.android.apps.maps', 'bri.delivery.brimobile',
    'com.bitsmedia.android.muslimpro', 'org.detikcom.rss', 'com.google.android.youtube', 'com.skype.raider',
    'com.supercell.clashofclans', 'com.microsoft.office.outlook', 'com.tiket.gits', 'com.tunaikita.cashloan',
    'com.google.android.apps.plus', 'com.gojek.driver.bike', 'com.indivara.jneone', 'com.google.android.music',
    'com.microsoft.office.powerpoint', 'com.altermyth.bima.tablet', 'com.tencent.mm', 'com.intsig.camscanner',
    'com.google.android.videos', 'com.nemo.vidmate', 'com.instagram.layout', 'src.com.bni',
    'com.android.bbk.lockscreen3', 'com.cleanmaster.mguard'
]

# Categorize applications
def categorize_app(package_name):
    return category_mapping.get(package_name, 'Unknown')
def competitor_apps(package_name):
    return competitor_mapping.get(package_name, 'None')

# # Create a DataFrame for the results
df_apps = pd.DataFrame({'app_package': applications})
df_apps['category'] = df_apps['app_package'].apply(categorize_app)

# print(df_apps)


In [20]:
applications= list(df_expanded['app_package'].unique())

In [113]:
competitor_apps= ['id.co.myhomecredit','com.julofinance.juloapp','com.indodana.app','com.kreditpintar',
                 'id.co.aeon.aeoncreditmerchant','id.investree','kreditplus'
                 ]

In [507]:
list_check= [a.lower().split(' ')[0] for a in ['Danamas', 'Investree', 'Amartha', 'DOMPET Kilat', 'Boost', 'TOKO MODAL',
        'Findaya', 'Modalku', 'KTA KILAT', 'Kredit Pintar', 'Maucash', 'Finmas',
        'KlikA2C', 'Akseleran', 'Ammana.id', 'PinjamanGO', 'KoinP2P', 'Pohondana',
        'MEKAR', 'AdaKami', 'ESTA KAPITAL FINTEK', 'KREDITPRO', 'FINTAG', 'RUPIAH CEPAT',
        'CROWDO', 'Indodana', 'JULO', 'Pinjamwinwin', 'DanaRupiah', 'Taralite',
        'Pinjam Modal', 'ALAMI', 'AwanTunai', 'Danakini', 'Singa', 'DANAMERDEKA',
        'EASYCASH', 'PINJAM YUK', 'FinPlus', 'UangMe', 'PinjamDuit', 'DANA SYARIAH',
        'BATUMBU', 'Cashcepat', 'klikUMKM', 'Pinjam Gampang', 'Cicil', 'Lumbungdana',
        '360 KREDI', 'SAMIR', 'Kredinesia', 'Pintek', 'ModalRakyat', 'SOLUSIKU',
        'Cairin', 'TrustIQ', 'KLIK KAMI', 'Duha SYARIAH', 'Invoila']]
# cols=list_check[4]
# cols='kredit'
[a for a in df_apps['app_package'] if 'grind' in a.lower()]

[]

In [506]:
# df_apps['app_package']

In [21]:
df_apps = pd.DataFrame({'app_package': applications})
df_apps['category'] = df_apps['app_package'].apply(categorize_app)


In [158]:
list_category= list(df_apps[df_apps['category']!='Unknown']['category'].value_counts().index)

In [125]:
competitor_mapping = dict(zip(competitor_apps,
        ['competitor' for a in range(len(competitor_apps))]
        
        ))

In [293]:
# df_apps[df_apps['category']!='Unknown']
with_index= pd.merge(
df_expanded,
df_apps,on='app_package',how='left')
with_index['competitor_flag']=with_index['app_package'].apply(competitor_apps)
with_competitor_apps= pd.concat([
df_expanded[~df_expanded['index'].isin(list(with_index[with_index['competitor_flag']!='None'].groupby(['index']).agg({'competitor_flag':'count'}).index))].drop_duplicates('index').set_index('index').drop(columns=['app_package']),
with_index[with_index['competitor_flag']!='None'].groupby(['index']).agg({'competitor_flag':'count'})],axis=0).fillna(0)


### bad rate competitor flag

In [297]:
# with_competitor_apps

In [176]:
# list(with_index[with_index['competitor_flag']=='None'][['index']].drop_duplicates()['index'])

In [179]:
# list(with_index[with_index['competitor_flag']=='None'][['index']].drop_duplicates()['index'])

In [247]:
# data[~data['index'].isin(df_expanded['index'].unique())]['']

## based on category

In [298]:
# with_index= pd.merge(
# df_expanded,
# df_apps,on='app_package',how='left')
# with_index['competitor_flag']=with_index['app_package'].apply(competitor_apps)
# with_index= pd.concat([with_index[with_index['competitor_flag']!='None'].groupby(['index']).agg({'competitor_flag':'count'}),
# with_index[with_index['competitor_flag']=='None'][['index']].drop_duplicates().set_index('index')],axis=0).fillna(0).sort_index()



In [299]:
cat_check= list_category[0]
cat_check

'Productivity'

In [248]:
list_apps_flag=[]
for cat_check in list_category:
    df_join= pd.merge(df_apps[df_apps['category']==cat_check],
    df_expanded,on='app_package',how='right').groupby(['index']).agg({'category':'count'}).rename(columns={'category':cat_check})
    list_apps_flag.append(df_join)
list_apps_flag= pd.concat(list_apps_flag,axis=1)

In [306]:
fe_apps_df= pd.concat([list_apps_flag,
with_competitor_apps],axis=1).astype(int)

In [307]:
fe_apps_df

,Productivity,Social Media,Financial,Entertainment,Utility,Photography,Shopping,Transportation,Communication,Media,Browser,Games,Navigation,competitor_flag
index,,,,,,,,,,,,,,
0,2,4,1,2,1,0,1,1,0,2,1,0,1,0
1,1,2,1,1,0,0,1,0,1,0,0,0,0,0
2,1,6,2,1,0,0,1,3,1,0,0,0,0,0
3,1,5,4,0,0,0,3,2,1,0,0,0,0,1
4,0,4,1,0,1,0,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41017,1,4,2,0,0,0,2,2,0,0,0,0,0,0
41018,0,2,1,0,1,0,4,0,2,2,0,0,0,0
41019,1,3,1,0,0,0,1,2,1,0,0,0,0,0


# adding mapping fintech 

In [479]:
list_check= [a.lower().split(' ')[0] for a in ['Danamas', 'Investree', 'Amartha', 'DOMPET Kilat', 'Boost', 'TOKO MODAL',
        'Findaya', 'Modalku', 'KTA KILAT', 'Kredit Pintar', 'Maucash', 'Finmas',
        'KlikA2C', 'Akseleran', 'Ammana.id', 'PinjamanGO', 'KoinP2P', 'Pohondana',
        'MEKAR', 'AdaKami', 'ESTA KAPITAL FINTEK', 'KREDITPRO', 'FINTAG', 'RUPIAH CEPAT',
        'CROWDO', 'Indodana', 'JULO', 'Pinjamwinwin', 'DanaRupiah', 'Taralite',
        'Pinjam Modal', 'ALAMI', 'AwanTunai', 'Danakini', 'Singa', 'DANAMERDEKA',
        'EASYCASH', 'PINJAM YUK', 'FinPlus', 'UangMe', 'PinjamDuit', 'DANA SYARIAH',
        'BATUMBU', 'Cashcepat', 'klikUMKM', 'Pinjam Gampang', 'Cicil', 'Lumbungdana',
        '360 KREDI', 'SAMIR', 'Kredinesia', 'Pintek', 'ModalRakyat', 'SOLUSIKU',
        'Cairin', 'TrustIQ', 'KLIK KAMI', 'Duha SYARIAH', 'Invoila']]
# cols=list_check[4]
# cols='kredit'
[a for a in list(df_expanded['app_package'].unique()) if 'casino' in a.lower()]

['com.crossfield.casinogame_bingo',
 'gameplay.casinomobile.cm88',
 'com.monarcy.GSCASINO',
 'gameplay.casinomobile.club9',
 'com.doubleugames.DoubleUCasino',
 'com.bingocasino.bingohd',
 'air.DBCasino',
 'com.hanamobiles.bingocasino',
 'com.zentertain.vegasstarcasino',
 'gameplay.casinomobile',
 'com.huuuge.casino.slots',
 'games.fatlion.las.vegas.free.minecraft.casino.exploration.lite.crafting.building.games.roblox.mcpe.free.build.city.hangover',
 'com.memoriki.fullhousecasino',
 'com.gonline.AppBravoCasino',
 'com.arenaon.cashtree.casino',
 'com.cynking.gaplecasino',
 'com.mobimobi.games.hot.casino.slots.game.mega.spin.slots.machine.game',
 'gameplay.casinomobile.imobet',
 'air.com.alisacasino.bingo',
 'com.winforfun88.livecasino',
 'id.mime.pro.dewacasino',
 'com.gsn.android.casino',
 'com.williamsinteractive.hotshotcasino',
 'com.slots.games.megajackpot.casinoparty4',
 'com.superluckycasino.epicjackpot.slots.vegas.android.free',
 'gameplay.casinomobile.ezebet',
 'com.nurigames.cas

In [428]:
list_app_pinjaman=['com.kreditpintar', 'com.kreditplus.kpm', 'com.laku6.ngebetkredit', 'com.oto.kreditkendaraan', 'com.rookie.hitungkreditrumah', 'id.kreditpasar.android.pasarkredit', 'com.mma.pasarkredit', 'com.kreditplus.eform',
 'com.kredit.ting.pinjamuang', 'com.kazoku.kreditmultiguna', 'com.rookie.hitungkredit', 'com.rookie.hitungkreditmobil', 'id.kreditpasar.android.kreditmart', 'com.pacificaraya.masskredit', 'com.mr.kreditpinjamantanpaagunan', 'com.kazoku.kredithp',
 'studio.code.kreditkta', 'id.kreditcepat.mobile', 'btx.keuangan.kredituang', 'com.rookie.hitungkreditbank', 'com.pinjamantunaikredit.anda', 'com.kreditpastimudah', 'co.tigbit.pinjamankreditcepat', 'cek.danacepat.cekkreditpinjaman',
 'com.jiehuahua.pinjamankredit', 'com.st18.pinjamankreditmandiri', 'com.infokom.bangkredit', 'gocash.danacepat.hantukreditan', 'com.aksimayamobile.kreditplus', 'com.cashkredit.android', 'com.ppdai.ayokredit2', 'com.cqtrf.kreditrupiah',
 'cplsoft.kreditpinjaman', 'com.mr.pinjamuang_kredittanpaagunan', 'com.kreditbpf', 'com.kredit.pinjamanmandri', 'com.wpinjamankredithptanpadp_6401205', 'com.kreditcepat.pinjamancepa', 'com.thunkable.android.ferybudud.Kartu_kredit_online',
 'com.yeskredit.loan', 'com.top.kredit', 'com.st17.hpkredit', 'com.majid.akreditasi', 'com.pinjaman.panduankredit', 'com.kreditmandiri.nasabah', 'com.kreditmandiri.mitra', 'com.kreditmandiri.droid',
 'id.co.loan.kreditpinjamanuang', 'com.pinjaman.kredithp', 'com.appmakr.kredithponline', 'com.ekredit.loan', 'com.kreditup.loan', 'com.appkartukredit', 'com.rupiah.pinjaman.gokredit', 'com.pengajuankredit.kartumandiri',
 'com.kredit.plus', 'com.info.tagihan.kartu.kredit', 'id.kta.kreditpinjaman.online', 'com.mkredit', 'com.fs.hitungkredit', 'com.unand.hitungkredit', 'com.uangtech.kreditkilat', 'com.rookie.hitungkreditmotor']+['we.pinjaman.rupiah',
 'com.pinjaman.uang', 'com.wPinjamandanatanpajaminanmemudahkantransaksipinjamandana_6129232', 'com.mobanker.pinjamanrp', 'com.gotunai.pinjaman', 'com.wpinjamanonline_6103889', 'com.pinjamango','com.dana.pinjaman',
 'com.pinjaman.rupiahnowofficial', 'tigbit.co.mantep_pinjamantepat', 'com.loan.gopinjaman', 'com.klgz.daikuan.pinjamandanacepat', 'kimo.pinjaman.cma', 'com.mr.kreditpinjamantanpaagunan', 'starwin.pinjamanmud.android',
 'com.pinjaman.mall', 'com.pinjaman.rupiah', 'com.pinjamantunaikredit.anda', 'com.fintopia.pinjaman', 'com.jrweid.rajapinjaman', 'com.cicilanbulanan.pinjamanuangrupiah', 'com.pinjaman.now',
 'co.tigbit.pinjamankreditcepat', 'cek.danacepat.cekkreditpinjaman', 'com.jiehuahua.pinjamankredit', 'com.st18.pinjamankreditmandiri', 'crom.pinjamansepuluh.hebat', 'pin.pinjamantunainya.pinjamanmodal', 'biro.pinjamuang.pinjamanmudah',
 'josh.nyilihduit.pinjamandanawoluh', 'cplsoft.kreditpinjaman', 'com.pinjaman.danakilat', 'com.pinjamantunaionlinebtn.mobile', 'com.jrwd.pinjamanuangtunai', 'com.rajapinjaman',
 'com.tabelangsuranpinjaman.anakemak', 'com.kredit.pinjamanmandri', 'com.wpinjamankredithptanpadp_6401205', 'com.kreditcepat.pinjamancepa', 'com.pinjaman.dana', 'com.pinjamantunaionlinebca.mobile', 'com.jrwd.mobilepinjaman', 'com.loan.pinjaman.dompet',
 'com.finlabtech.pinjaman', 'com.sakucepiatindustri.pinjamanuang', 'id.co.pinjaman.kilat.loan', 'com.dompetkilat.pinjaman', 'com.rumah.pinjaman', 'com.homepinjaman', 'com.xinhe.pinjaman', 'com.loan1.cepatpinjaman', 'uuang.pinjaman.rupiah',
 'com.kta.bripinjaman', 'com.pinjamanonline.kurbri', 'com.pinjaman.panduankredit', 'com.kurbri.pinjamankurbriguide', 'com.hstech.inc.pinjamanuangonline', 'com.tl.pinjamanduit', 'id.co.loan.kreditpinjamanuang', 'com.xzkj.pinjamancash', 'com.bankonline.pinjamandankur2018', 'com.done.jammas.pinjamanonlinetanpajaminan',
 'com.pinjaman.kredithp', 'com.done.jammas.pinjamantanpajaminanonline', 'id.pinjamanmud.android', 'com.pinjaman.ktacepat', 'com.super.pinjaman', 'com.rupiah.pinjaman.gokredit', 'com.mr.pinjamanonlinetanpajaminan', 'com.tke.pinjaman', 'id.co.pinjaman.trie.kta', 'id.kta.kreditpinjaman.online',
 'id.indosejah.pinjamanuang.online', 'com.three.pinjamanonlinemandiri', 'com.easypinjaman.easy', 'com.pinjaman.uangdana', 'id.wystudio.pinjamanonline', 'com.haiuang.pinjaman', 'com.carirupiah.pinjaman', 'com.pinjaman.sup', 'com.tl.sakupinjaman',
 'com.programindo.pinjamanonlinemandiri']+['we.pinjaman.rupiah',
 'id.danarupiah.weshare.jiekuan', 'com.newzqxq.halorupiah',
 'com.finstar.drupiah', 'com.infincash.gorupiah',
 'com.rupiahq.cash', 'com.ipogroup.rajarupiah',
 'com.ipogroup.kamirupiah', 'com.rupiah.uang',
 'com.pinjaman.rupiahnowofficial', 'com.klgz.daikuan.rupiahkilat',
 'com.hhcfintech.rupiaheasy', 'com.cashrupiah.cash',
 'com.k93829444yinni.UangRupiah', 'com.ipogroup.ayorupiah',
 'com.weifeng.nowrupiah', 'com.rupiahtemanonline',
 'com.mma.myrupiah', 'com.pinjaman.rupiah',
 'com.cicilanbulanan.pinjamanuangrupiah', 'com.cqtrf.kreditrupiah',
 'com.jrweid.rupiahteman', 'com.ipogroup.rupiahcepat',
 'com.beebird.akurupiah', 'com.ipogroup.rupiahkilat',
 'com.raja.rupiah', 'com.pinjam.rupiah',
 'com.minirupiah.mr', 'com.newrupiah.uang',
 'com.wHaloRupiahPinjamanUangRupiahDanaCepatdanMudah_6498725', 'com.ILJOVITA.KursRupiahHariIni',
 'uuang.pinjaman.rupiah', 'com.loan.cash.credit.easy.kilat.cepat.pinjam.uang.dana.rupiah',
 'com.rupiah.ayo', 'com.rupiah.kita',
 'com.cashkilatindustri.sakudanarupiah', 'com.sakucepiatindustri.kamirdanarupiah', 'com.rupiahmart.android',
 'com.rupiah.oke', 'com.rupiah.bijak',
 'com.rupiah.pinjaman.gokredit', 'com.uangtech.erupiah',
 'com.pinjama.rupiah', 'com.rupiahhero.tempest',
 'com.carirupiah.pinjaman', 'com.newzqxq.harorupiah',
 'com.xinhe.yesrupiah', 'com.cepatmoney.rajarupiahcepat',
 'com.cashkilatindustri.rupiahzone', 'com.raja.rupiaha']

In [482]:
gamble_online= ['com.boyaa.gaple.qiuqiu',
 'com.funplay.indo.dominoqiuqiu',
 'com.domino.qiuqiu',
 'com.cynking.qiuqiu','com.fortune.sim.game.cash', 'com.huuuge.casino.slots',
 'com.playstudios.popslots',
 'com.murka.slotsera',
 'com.bigfishgames.jackpotcityslotsf2pgoogle',
 'com.mobimobi.games.hot.casino.slots.game.mega.spin.slots.machine.game',
 'com.murka.infinityslots',
 'com.infiapps.dolphinslots',
 'com.india.freeslots',
 'com.slots.games.megajackpot.casinoparty4',
 'com.sagafun.vegasslots',
 'com.superluckycasino.epicjackpot.slots.vegas.android.free',
 'com.vegas.slots777',
 'com.hotcasino.vegas.slots.grandwin.free',
 'com.superluckycasino.viponly.slots.vegas.android.free','com.crossfield.casinogame_bingo',
 'gameplay.casinomobile.cm88',
 'com.monarcy.GSCASINO',
 'gameplay.casinomobile.club9',
 'com.doubleugames.DoubleUCasino',
 'com.bingocasino.bingohd',
 'air.DBCasino',
 'com.hanamobiles.bingocasino',
 'com.zentertain.vegasstarcasino',
 'gameplay.casinomobile',
 'com.huuuge.casino.slots',
 'games.fatlion.las.vegas.free.minecraft.casino.exploration.lite.crafting.building.games.roblox.mcpe.free.build.city.hangover',
 'com.memoriki.fullhousecasino',
 'com.gonline.AppBravoCasino',
 'com.arenaon.cashtree.casino',
 'com.cynking.gaplecasino',
 'com.mobimobi.games.hot.casino.slots.game.mega.spin.slots.machine.game',
 'gameplay.casinomobile.imobet',
 'air.com.alisacasino.bingo',
 'com.winforfun88.livecasino',
 'id.mime.pro.dewacasino',
 'com.gsn.android.casino',
 'com.williamsinteractive.hotshotcasino',
 'com.slots.games.megajackpot.casinoparty4',
 'com.superluckycasino.epicjackpot.slots.vegas.android.free',
 'gameplay.casinomobile.ezebet',
 'com.nurigames.casinolive',
 'com.roulette.casino.free',
 'com.hotcasino.vegas.slots.grandwin.free',
 'com.superluckycasino.viponly.slots.vegas.android.free',
 'com.playtika.caesarscasino',
 'com.casinovr.casinovrpoker']

In [487]:
# list_casino= ['com.crossfield.casinogame_bingo',
#  'gameplay.casinomobile.cm88',
#  'com.monarcy.GSCASINO',
#  'gameplay.casinomobile.club9',
#  'com.doubleugames.DoubleUCasino',
#  'com.bingocasino.bingohd',
#  'air.DBCasino',
#  'com.hanamobiles.bingocasino',
#  'com.zentertain.vegasstarcasino',
#  'gameplay.casinomobile',
#  'com.huuuge.casino.slots',
#  'games.fatlion.las.vegas.free.minecraft.casino.exploration.lite.crafting.building.games.roblox.mcpe.free.build.city.hangover',
#  'com.memoriki.fullhousecasino',
#  'com.gonline.AppBravoCasino',
#  'com.arenaon.cashtree.casino',
#  'com.cynking.gaplecasino',
#  'com.mobimobi.games.hot.casino.slots.game.mega.spin.slots.machine.game',
#  'gameplay.casinomobile.imobet',
#  'air.com.alisacasino.bingo',
#  'com.winforfun88.livecasino',
#  'id.mime.pro.dewacasino',
#  'com.gsn.android.casino',
#  'com.williamsinteractive.hotshotcasino',
#  'com.slots.games.megajackpot.casinoparty4',
#  'com.superluckycasino.epicjackpot.slots.vegas.android.free',
#  'gameplay.casinomobile.ezebet',
#  'com.nurigames.casinolive',
#  'com.roulette.casino.free',
#  'com.hotcasino.vegas.slots.grandwin.free',
#  'com.superluckycasino.viponly.slots.vegas.android.free',
#  'com.playtika.caesarscasino',
#  'com.casinovr.casinovrpoker']

In [438]:
list_fintech = ['id.investree','id.co.danamas.v5','id.co.modalku.danacepat',
 'com.pinjamango','com.crowdo.p2pconnect','com.tl.bostunai','com.firestorm.sea.cashcash',
'com.finlabtech.cashbon','com.finlabtech.pinjaman','id.kilatfintech.android','com.cd.DanaBerkat',
 'id.com.la.danaku', 'com.xinhe.danaku', 'com.haiqian.danaku','com.lingyue.idnEasycash',
 'com.firestorm.sea.easycash', 'com.lingyue.idnEasycash_1', 'com.fintopia.idnEasycash'
]

In [889]:
fe_apps_upd['competitor_flag'].value_counts()

0.0    32691
1.0     6049
2.0     1096
3.0      194
4.0       58
5.0       14
6.0        5
7.0        1
8.0        1
Name: competitor_flag, dtype: int64

In [495]:
fe_apps_upd= pd.concat([fe_apps_df,
df_expanded[df_expanded['app_package'].isin(list_fintech)].groupby(['index']).agg({'app_package':'count'}).rename(columns={'app_package':'fintech'}),
                        df_expanded[df_expanded['app_package'].isin(gamble_online)].groupby(['index']).agg({'app_package':'count'}).rename(columns={'app_package':'judol_apps'}),
df_expanded[df_expanded['app_package'].isin(list_app_pinjaman)].groupby(['index']).agg({'app_package':'count'}).rename(columns={'app_package':'pinjaman'}),
                       fe_apps_upd.T.sum().astype(int).rename('total_installed')
                       ],axis=1).fillna(0).astype(int)

In [508]:
# Information about data type, missing values, unique sample
def data_information(df):
    data_info = pd.DataFrame({'Data_Type':df.dtypes,
                              'N_Missing':df.isnull().sum(),
                              '%_Missing':["{0:.0%}".format(x) for x in df.isnull().sum()/len(df)], 
                              'N_Unique':df.nunique().values,
                              'Sample':list(zip(df.sample(5).values[0], df.sample(5).values[4]))})
    return data_info

def cat_or_num(df):
    cat_cols = df.select_dtypes(["object", "category"]).columns.tolist()
    num_but_cat = [col for col in df.select_dtypes(["float", "integer"]).columns if df[col].nunique() <= 15]
    cat_but_many = [col for col in df.select_dtypes(["object", "category"]).columns if df[col].nunique() > 15]
    
    cat_cols = [col for col in cat_cols if col not in cat_but_many]

    num_cols = df.select_dtypes(["float", "integer"]).columns
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {df.shape[0]}")
    print(f"Variables: {df.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    print(f'cat_but_many: {len(cat_but_many)}')

    return cat_cols, num_cols, num_but_cat, cat_but_many
    
def histogram(df, columns):
    
    n_rows = math.ceil(len(columns)/3)
    col_name = columns
    label = 'TARGET'
    
    fig, axes = plt.subplots(n_rows, 3, figsize=(15,3*n_rows))

    for name, ax in zip(col_name, axes.flatten()):

        sns.distplot(df[df[label]==0][name], ax=ax, label='non-default', color='#5DBAE8')
        sns.distplot(df[df[label]==1][name], ax=ax, label='default', color='#D71B3B')
        ax.legend(prop={'size': 12})
        ax.set_title('Histogram ' + name)  
        
    plt.tight_layout()
    plt.show()

In [749]:
list_app_installed = list(fe_apps_upd.columns)

In [759]:
fe_apps_upd= pd.merge(fe_apps_upd.reset_index(),
data[['index']],how='right',on='index').sort_index()

In [782]:
df=fe_apps_upd.copy()
def compute_ratio(numerator, denominator):
    with np.errstate(divide='ignore', invalid='ignore'):
        return np.where(denominator != 0, numerator / denominator, np.nan)

# Create new columns for ratios
for i in range(len(list_app_installed)):
    for j in range(len(list_app_installed)):
        if i != j:
            num_col = list_app_installed[i]
            denom_col = list_app_installed[j]
            ratio_col = f'ratio_{num_col}_{denom_col}'
            df[ratio_col] = compute_ratio(df[num_col], df[denom_col])

fe_apps_upd_ratio=df.copy()[[a for a in df.columns if a not in list_app_installed]]

<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_ratio(df[num_col], df[denom_col])
<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_ratio(df[num_col], df[denom_col])
<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead

<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_ratio(df[num_col], df[denom_col])
<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_ratio(df[num_col], df[denom_col])
<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead

<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_ratio(df[num_col], df[denom_col])
<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_ratio(df[num_col], df[denom_col])
<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead

<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_ratio(df[num_col], df[denom_col])
<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_ratio(df[num_col], df[denom_col])
<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead

<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_ratio(df[num_col], df[denom_col])
<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_ratio(df[num_col], df[denom_col])
<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead

<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_ratio(df[num_col], df[denom_col])
<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_ratio(df[num_col], df[denom_col])
<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead

<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_ratio(df[num_col], df[denom_col])
<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_ratio(df[num_col], df[denom_col])
<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead

<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_ratio(df[num_col], df[denom_col])
<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_ratio(df[num_col], df[denom_col])
<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead

<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_ratio(df[num_col], df[denom_col])
<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_ratio(df[num_col], df[denom_col])
<ipython-input-782-517deb8f5907>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead

In [775]:
df=fe_apps_upd.copy()
def compute_sum(column1, column2):
    return column1 + column2

# Create new columns for ratios
for i in range(len([a for a in list_app_installed if 'total' not in a ])):
    for j in range(len([a for a in list_app_installed if 'total' not in a ])):
        if i != j:
            num_col = [a for a in list_app_installed if 'total' not in a ][i]
            denom_col = [a for a in list_app_installed if 'total' not in a ][j]
            ratio_col = f'sum_{num_col}_{denom_col}'
            df[ratio_col] = compute_sum(df[num_col], df[denom_col])

fe_apps_upd_sum=df.copy()[[a for a in fe_apps_upd_sum.columns if a not in list_app_installed]]

<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_sum(df[num_col], df[denom_col])
<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_sum(df[num_col], df[denom_col])
<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To

<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_sum(df[num_col], df[denom_col])
<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_sum(df[num_col], df[denom_col])
<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To

<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_sum(df[num_col], df[denom_col])
<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_sum(df[num_col], df[denom_col])
<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To

<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_sum(df[num_col], df[denom_col])
<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_sum(df[num_col], df[denom_col])
<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To

<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_sum(df[num_col], df[denom_col])
<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_sum(df[num_col], df[denom_col])
<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To

<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_sum(df[num_col], df[denom_col])
<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_sum(df[num_col], df[denom_col])
<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To

<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_sum(df[num_col], df[denom_col])
<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ratio_col] = compute_sum(df[num_col], df[denom_col])
<ipython-input-775-6a5f73d15992>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To

In [765]:
# fe_call_list_ratio=  pd.merge(fe_apps_upd,df[[a for a in df.columns if a not in fe_call_list_cols]],on='index').head(2)

## join apps installed feature

In [900]:
final_fe_apps_installed= pd.concat([fe_apps_upd_ratio.set_index('index'),
fe_apps_upd.set_index('index'),
fe_apps_upd_sum.set_index('index')],axis=1)

In [904]:
final_fe_apps_installed.reset_index().to_csv('fe_apps_installed.csv',index=False)

# adding call logs

In [610]:
import json 

In [686]:
app_cal_log= ['percent_incoming_nighttime', 'percent_outgoing_daytime',
       'duration_incoming_daytime', 'duration_outgoing_daytime',
       'percent_incoming_daytime', 'percent_other_calls',
       'duration_outgoing_nighttime', 'percent_outgoing_nighttime',
       'total_calls', 'duration_incoming_nighttime']

In [664]:
# data['ph_call_log_stats'][0]
# df = pd.DataFrame(data)

# Convert JSON column from string to dictionary

df=data[data['ph_call_log_stats'].notnull()].copy().set_index('id_credit')
df['ph_call_log_stats'] = df['ph_call_log_stats'].apply(json.loads)

# Normalize JSON column to separate columns
df_normalized = pd.json_normalize(df['ph_call_log_stats'])

# Concatenate original DataFrame with normalized DataFrame
# df_final = pd.concat([df[['index','ph_call_log_stats']].drop(columns='ph_call_log_stats'), df_normalized], axis=1)

In [683]:
# data.shape

In [670]:
df=data.copy()
df = df.set_index('id_credit')

# Drop rows where 'ph_call_log_stats' is null
df = df[df['ph_call_log_stats'].notnull()]

# Convert JSON column from string to dictionary
def safe_json_loads(json_str):
    try:
        return json.loads(json_str)
    except (TypeError, json.JSONDecodeError):
        return {}

df['ph_call_log_stats'] = df['ph_call_log_stats'].apply(safe_json_loads)

# Normalize JSON column to separate columns
df_normalized = pd.json_normalize(df['ph_call_log_stats'])

# Concatenate original DataFrame with normalized DataFrame
df_final = pd.concat([df.drop(columns='ph_call_log_stats'), df_normalized], axis=1)

In [695]:
fe_call_list= pd.merge(data[['index']],df_final[app_cal_log+['index']],on='index',how='left')

In [700]:

fe_call_list_cols=['percent_incoming_nighttime',
 'percent_outgoing_daytime',
 'duration_incoming_daytime',
 'duration_outgoing_daytime',
 'percent_incoming_daytime',
 'percent_other_calls',
 'duration_outgoing_nighttime',
 'percent_outgoing_nighttime',
 'total_calls',
 'duration_incoming_nighttime']

### FE the columns

In [794]:
df=fe_call_list.copy()
def compute_ratio(numerator, denominator):
    with np.errstate(divide='ignore', invalid='ignore'):
        return np.where(denominator != 0, numerator / denominator, np.nan)

# Create new columns for ratios
for i in range(len(fe_call_list_cols)):
    for j in range(len(fe_call_list_cols)):
        if i != j:
            num_col = fe_call_list_cols[i]
            denom_col = fe_call_list_cols[j]
            ratio_col = f'ratio_{num_col}_{denom_col}'
            df[ratio_col] = compute_ratio(df[num_col], df[denom_col])

fe_call_list_ratio=  df.copy()[[a for a in df.columns if a not in fe_call_list_cols]]

In [795]:
df=fe_call_list.copy()
def compute_sum(column1, column2):
    return column1 + column2

# Create new columns for summation
for i in range(len(fe_call_list_cols)):
    for j in range(len(fe_call_list_cols)):
        if i != j:
            col1 = fe_call_list_cols[i]
            col2 = fe_call_list_cols[j]
            sum_col = f'sum_{col1}_{col2}'
            df[sum_col] = compute_sum(df[col1], df[col2])
fe_call_list_sum= df.copy()[[a for a in df.columns if a not in fe_call_list_cols]]

In [802]:
final_fe_callist= pd.concat([fe_call_list.set_index('index'),
fe_call_list_sum.set_index('index'),
fe_call_list_ratio.set_index('index')],axis=1).sort_index()

In [804]:
# final_fe_callist.head(5)

# adding main dataframe

In [707]:

data['fb_last_updated_date']= pd.to_datetime(data['fb_last_updated_date'])
data['de_date_joined']= pd.to_datetime(data['de_date_joined'])


In [593]:
def safe_to_datetime(date_str):
    try:
        # Convert to datetime and remove timezone if present
        dt = pd.to_datetime(date_str, errors='coerce')
        if dt is not pd.NaT and dt.tzinfo is not None:
            dt = dt.tz_localize(None)  # Remove timezone information
        return dt
    except Exception as e:
        return pd.NaT

# Convert columns to datetime
data['fb_last_updated_date'] = data['fb_last_updated_date'].apply(safe_to_datetime)
data['de_date_joined'] = data['de_date_joined'].apply(safe_to_datetime)

In [ ]:
data

In [608]:
data['fb_latest_update']= (data['de_date_joined'] - data['fb_last_updated_date']).dt.days
data['r_salary_cnt_children']=data['de_monthly_salary']/(data['de_children']+1)

In [850]:
additional_fe=['fb_latest_update','r_salary_cnt_children','ph_other_device_info_upd']

In [859]:
data.groupby(['de_gender','fb_gender']).agg({'index':'count'})/data.shape[0]*100

index
de_gender fb_gender           
1         female      1.109161
          male       57.798255
2         female     36.665692
          male        0.738628

In [860]:
cols_to_drop=['de_gender','ph_other_device_info','ph_country_code','fb_dob','fb_gender']

In [832]:
data['ph_other_device_info_upd']=[str(a).strip() for a in data['ph_other_device_info'] if a != np.nan]
list_top10= list((data['ph_other_device_info_upd'].value_counts()/data['ph_other_device_info_upd'].value_counts().sum()*100).head(9).index)
data.loc[~(data['ph_other_device_info_upd'].isin(list_top10))&(data['ph_other_device_info_upd']!='nan'),'ph_other_device_info_upd']='OTHERS'
data.loc[data['ph_other_device_info'].isnull(),'ph_other_device_info_upd']=np.nan

In [879]:
data['ph_other_device_info_upd'].value_counts(dropna=False)

Xiaomi       10008
samsung       8869
OPPO          7474
xiaomi        3756
OTHERS        2949
vivo          2460
asus          2424
Lenovo        1483
Sony           664
Smartfren      639
NaN            296
Name: ph_other_device_info_upd, dtype: int64

In [866]:
cols_index=['de_date_joined','index','flag_bad']

# join all final FE data for modelling 

In [874]:
data_final_modelling= pd.concat([data[[a for a in cols_pred if a not in cols_to_drop]+cols_index+additional_fe].set_index('index'),
final_fe_apps_installed,
final_fe_callist],axis=1).reset_index()

In [884]:
cols_to_drop

['de_gender', 'ph_other_device_info', 'ph_country_code', 'fb_dob', 'fb_gender']

In [883]:
cols_index+['id_credit']

['de_date_joined', 'index', 'flag_bad', 'id_credit']

In [876]:
data_final_modelling['id_credit']=data_final_modelling['index']

In [880]:
data_final_modelling.to_csv('data_modelling.csv',index=False)

In [852]:
# Data Information
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

data_information(data[cols_pred])

,Data_Type,N_Missing,%_Missing,N_Unique,Sample
de_age,int64,0,0%,50,"(40, 27)"
de_gender,int64,0,0%,2,"(1, 2)"
de_date_joined,datetime64[ns],0,0%,192,"(2018-06-18 00:00:00, 2018-05-24 00:00:00)"
de_num_friends,int64,0,0%,4531,"(143, 1430)"
de_monthly_salary,int64,0,0%,40,"(5000000, 3000000)"
de_employment_type,int64,0,0%,4,"(1, 3)"
de_employment_duration,int64,0,0%,52,"(51, 38)"
de_accomodation_type,int64,0,0%,1,"(4, 4)"
de_education,int64,0,0%,5,"(3, 1)"
de_marital_status,int64,0,0%,4,"(2, 2)"
